In [106]:
import pandas as pd
import sklearn as sk
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

### **Algoritmos a probar**

**Linear**
* Logistic Regression
* Linear SVC (regularized - unregularized)

**Non-linear**
* Poly SVC
* rbf SVC

**otros**
* naïve Bayes
* Nearest Neighbors Classification
* Stochastic Gradient Descent
* Gaussian Process Classification
* Decision Trees

**Ensambles**
* Random Forests
* AdaBoost
* Gradient Tree Boosting

**Neural Network**
* Multi-layer Perceptron

### Pre-procesamiento de datos

In [107]:
# Cargo datos parte1
avisos_parte1 =  pd.read_csv('../data/datos_navent_fiuba/fiuba_6_avisos_detalle.csv')
postulaciones_parte1 = pd.read_csv('../data/datos_navent_fiuba/fiuba_4_postulaciones.csv')
vistas_parte1 = pd.read_csv('../data/datos_navent_fiuba/fiuba_3_vistas.csv')
edadYGenero_parte1 = pd.read_csv('../data/datos_navent_fiuba/fiuba_2_postulantes_genero_y_edad.csv')
educacion_parte1 = pd.read_csv('../data/datos_navent_fiuba/fiuba_1_postulantes_educacion.csv')

# Cargo datos hasta 15_4
avisos_hasta15_4 =  pd.read_csv('./data/Fiuba hasta 15 Abril/fiuba_6_avisos_detalle.csv')
postulaciones_hasta15_4 = pd.read_csv('data/Fiuba hasta 15 Abril/fiuba_4_postulaciones.csv')
vistas_hasta15_4 = pd.read_csv('data/Fiuba hasta 15 Abril/fiuba_3_vistas.csv')
edadYGenero_hasta15_4 = pd.read_csv('data/Fiuba hasta 15 Abril/fiuba_2_postulantes_genero_y_edad.csv')
educacion_hasta15_4 = pd.read_csv('data/Fiuba hasta 15 Abril/fiuba_1_postulantes_educacion.csv')

# Concateno la info
avisos = pd.concat([avisos_parte1, avisos_hasta15_4])
postulaciones = pd.concat([postulaciones_parte1, postulaciones_hasta15_4])
vistas = pd.concat([vistas_parte1, vistas_hasta15_4])
edadYGenero = pd.concat([edadYGenero_parte1, edadYGenero_hasta15_4])
educacion = pd.concat([educacion_parte1, educacion_hasta15_4])

# Elimino df innecesarios
del(avisos_hasta15_4,
    avisos_parte1,
    postulaciones_hasta15_4,
    postulaciones_parte1,
    vistas_hasta15_4,
    vistas_parte1,
    edadYGenero_hasta15_4,
    edadYGenero_parte1,
    educacion_hasta15_4,
    educacion_parte1)

# Guardo una lista con los datasets
data = [avisos, postulaciones, vistas, edadYGenero, educacion]

# Reseteo los index
for df in data:
  df.reset_index(drop=True, inplace=True)

# Elimino duplicados
for df in data:
  if(df.duplicated().any().any()):
    df.drop_duplicates(inplace=True)

# Remuevo columnas innecesarias
avisos.drop(columns=['idpais','ciudad','mapacalle'], inplace=True)

# Renombro columnas
educacion.rename(columns={'nombre':'nivel'}, inplace=True)
vistas.rename(columns={'idAviso':'idaviso','timestamp':'fechavista'}, inplace=True)

In [108]:
# Veo que set y columnas tiene datos faltantes
datos = {'avisos': avisos, 'postulaciones': postulaciones, 'vistas': vistas, 'edadYGenero': edadYGenero, 'educacion': educacion}
for name, df in datos.items():
  cols_con_nulls = df.isnull().any()
  print(f"[{name}]",cols_con_nulls.any())
  if cols_con_nulls.any():
    cols = df.columns[cols_con_nulls.values][0]
    print('  columnas ->',cols)
    for column in [cols]:
      print(f'  tasa[{column}]:', len(df[df[[column]].isnull().values])/len(df))
  print()

[avisos] True
  columnas -> denominacion_empresa
  tasa[denominacion_empresa]: 0.0003048006096012192

[postulaciones] False

[vistas] False

[edadYGenero] True
  columnas -> fechanacimiento
  tasa[fechanacimiento]: 0.04809377846111661

[educacion] False



In [109]:
# AVISOS
# Analizo cols con nulls
if(avisos.isna().any().any()):
  cols_con_nulls = avisos.columns[avisos.isna().any().values]
  for col in cols_con_nulls:
    print(col)
    print(avisos[col].isna().value_counts())

# Elimino filas con datos faltantes
if(avisos.isna().any().any()):
  avisos.dropna(inplace=True)

denominacion_empresa
False    19679
True         6
Name: denominacion_empresa, dtype: int64


In [110]:
# EDADYGENERO
# Elimino filas con datos faltantes
if(edadYGenero.isna().any().any()):
  edadYGenero.dropna(inplace=True)

# Convierto columnas con fechas a datetime
edadYGenero['fechanacimiento'] = pd.to_datetime(edadYGenero.fechanacimiento, format='%Y/%m/%d', errors='coerce')
if(edadYGenero.isna().any().any()):
  print(edadYGenero.isna().any())
  edadYGenero.dropna(inplace=True)

# Agrego columna edad
anio_actual = pd.to_datetime('today').year
edadYGenero['edad'] = edadYGenero.fechanacimiento.map(lambda fecha: anio_actual - fecha.year).astype(int)

# Elimino la columna fechanacimiento
edadYGenero.drop(columns=['fechanacimiento'], inplace=True)

# Analizo los datos de la columna 'sexo'
print(edadYGenero.sexo.value_counts())
for value in edadYGenero.sexo.value_counts().index:
  if((value != 'FEM') & (value != 'MASC')):
    print(f'tasa [{value}]/total:',len(edadYGenero.sexo[edadYGenero.sexo == value])/len(edadYGenero))

# Reemplazo esos valores por valor moda
moda = edadYGenero.sexo.mode()[0]
print('mode',moda)
edadYGenero.sexo.replace(['NO_DECLARA','0.0'], moda, inplace=True)
print(edadYGenero.sexo.value_counts())

# Elimino filas con id duplicado
if(edadYGenero.idpostulante.duplicated().any()):
  print('total:',len(edadYGenero),'| duplicados:',len(edadYGenero[edadYGenero.idpostulante.duplicated()]))
  print('tasa:',len(edadYGenero[edadYGenero.idpostulante.duplicated()])/len(edadYGenero))
  edadYGenero.drop(edadYGenero[edadYGenero.idpostulante.duplicated()].index, inplace=True)

# Cambio valores categóricos a columnas
genero = {'FEM': 0,'MASC': 1}
edadYGenero['sexo'] = edadYGenero.sexo.map(lambda x: genero[x])

idpostulante       False
fechanacimiento     True
sexo               False
dtype: bool
FEM           204023
MASC          183814
NO_DECLARA       943
0.0                1
Name: sexo, dtype: int64
tasa [NO_DECLARA]/total: 0.002425530054195035
tasa [0.0]/total: 2.5721421571527415e-06
mode FEM
FEM     204967
MASC    183814
Name: sexo, dtype: int64
total: 388781 | duplicados: 146
tasa: 0.00037553275494430025


In [111]:
# EDUCACIÓN
# Analizo valores
nivel_counts = educacion.nivel.value_counts()
print('[nivel]')
for val in nivel_counts.index:
  print(val,' '*(17-len(val)),nivel_counts[val],' '*(7-len(str(nivel_counts[val]))),nivel_counts[val]/len(educacion))

# Reemplazo valor 'Otro' por valor moda
mode = educacion.nivel.mode()[0]
educacion.nivel.replace('Otro', mode, inplace=True)
print()
print('[estado]')
print(educacion.estado.value_counts())

# Convierto valores categóricos a columnas booleanas
nivel = {
  'Doctorado': 6,
   'Master': 5,
   'Posgrado': 4,
   'Universitario': 3,
   'Terciario/Técnico': 2,
   'Secundario': 1
  }
educacion['nivel'] = educacion.nivel.map(lambda x: nivel[x])

estado = {'Graduado': 3,'En Curso': 2, 'Abandonado': 1}
educacion['estado'] = educacion.estado.map(lambda x: estado[x])

[nivel]
Universitario      207752   0.3653668934176556
Secundario         199375   0.35063452758647373
Terciario/Técnico  92632    0.1629089783543084
Otro               43225    0.07601844491498597
Posgrado           16822    0.029584321118794537
Master             8299     0.01459518968998192
Doctorado          507      0.0008916449177998354

[estado]
Graduado      370213
En Curso      149388
Abandonado     49011
Name: estado, dtype: int64


In [112]:
educacion.estado.isna().any()

False

In [113]:
# Busco el nivel de estudio completado más alto
educacion_grouped = educacion.groupby('idpostulante')
# esto está mal
usuarios_edu = educacion_grouped.agg({'nivel': 'max','estado': lambda x: np.count_nonzero(x == 3)}).reset_index()
usuarios_edu.rename(columns={'nivel':'max_nivel_ed','estado':'cant_estudios'}, inplace=True)

In [159]:
# debería funcionar pero no
[max([val for (ix, val) in enumerate(cols.nivel) if cols.estado[ix] == 3], default=0) for (idx,cols) in educacion_grouped.__iter__()]

KeyError: 0

In [42]:
# Analizo las fechas de postulaciónes y vistas
fechas = pd.to_datetime(postulaciones.fechapostulacion)
print('[posts]', ' min:', fechas.min(), '| max:', fechas.max())
fechas = pd.to_datetime(vistas.fechavista)
print('[vistas]', 'min:', fechas.min(), '| max:', fechas.max())

[posts] min: 2018-01-15 00:00:01 | max: 2018-04-17 10:42:39
[visats] min: 2018-02-23 18:38:10.808000 | max: 2018-04-16 19:04:16.873000


In [ ]:
#vistas.groupby(['idpostulante','idaviso']).count()
vistas.loc[(vistas.idpostulante == '0zB026d') & (vistas.idaviso == 1112358250),:]

In [ ]:
# Unifico vistas y postulaciones
vistas_y_postulaciones = pd.merge(vistas, postulaciones, on=['idpostulante','idaviso'], how='outer')

# Renombro columnas y reordeno
vistas_y_postulaciones.rename(columns={'fechavista':'vista','fechapostulacion':'postulacion'}, inplace=True)
vistas_y_postulaciones = vistas_y_postulaciones[['idaviso','idpostulante','vista','postulacion']]

# Cambio valores de fechas por 1 si hay fecha y 0 si es NaN
vistas_y_postulaciones.vista = vistas_y_postulaciones.vista.notnull().astype('int')
vistas_y_postulaciones.postulacion = vistas_y_postulaciones.postulacion.notnull().astype('int')

In [ ]:
postulaciones.groupby(['idpostulante','idaviso']).count()[lambda x: x.fechapostulacion == 5]

In [ ]:
usr_pos = postulaciones.idpostulante.unique()
usr_info = edadYGenero.idpostulante.unique()
print(len(usr_pos),len(usr_info))